<a href="https://colab.research.google.com/github/CptK1ng/dmc2019/blob/master/notebooks/evaluate_automl_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# evaluate results from AutoML

In [0]:
import numpy as np
import pandas as pd
from sklearn import metrics
from IPython.display import display

In [0]:
!wget -nc -q --show-progress https://www.dropbox.com/s/6m8iq9ogpzmu7vx/train_new.csv?dl=1 -O train_new.csv
!wget -nc -q --show-progress https://www.dropbox.com/s/tjpkc45oqn3uv8s/val_new.csv?dl=1 -O val_new.csv
!wget -nc -q --show-progress https://www.dropbox.com/s/hbd6nzgwlnevu4x/test.csv?dl=1 -O test.csv
!wget -nc -q --show-progress https://www.dropbox.com/s/z4nowycqk0fgu0u/pred_test_1.csv?dl=1 -O pred_test_1.csv
!wget -nc -q --show-progress https://www.dropbox.com/s/1auugm1h0yzliqo/pred_val_1.csv?dl=1 -O pred_val_1.csv
!wget -nc -q --show-progress https://www.dropbox.com/s/t8fmybg7212yg1a/pred_val_2.csv?dl=1 -O pred_val_2.csv

In [66]:
# download datasets
train_df = pd.read_csv("train_new.csv", sep="|")
val_df = pd.read_csv("val_new.csv", sep="|")
test_df = pd.read_csv("test.csv", sep="|")

#download predictions
test_pred_df = pd.read_csv("pred_test_1.csv", sep=",")
test_pred_df["fraud_pred"] = np.where(test_pred_df["fraud_1_score"] > 0.5, 1, 0)
val_pred1_df = pd.read_csv("pred_val_1.csv", sep=",").rename(columns={'fraud':'fraud_pred'})
val_pred2_df = pd.read_csv("pred_val_2.csv", sep=",").rename(columns={'fraud':'fraud_pred'})

# calculate pred_thresh for classification based on 0.71 treshold
treshold = 0.71
test_pred_df["fraud_pred_thresh"] = np.where(test_pred_df["fraud_1_score"] > treshold, 1, 0)
val_pred1_df["fraud_pred_thresh"] = np.where(val_pred1_df["fraud_1_score"] > treshold, 1, 0)
val_pred2_df["fraud_pred_thresh"] = np.where(val_pred2_df["fraud_1_score"] > treshold, 1, 0)

# as we can see the following are having different X values / another sorting:
display(val_df.head(2))
display(val_pred1_df.head(2))

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
0,6,1504,89.37,8,5,3,0.018617,0.059422,0.285714,0
1,6,128,63.32,5,4,4,0.226562,0.494688,0.172414,0


,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud_pred,fraud_0_score,fraud_1_score,fraud_pred_thresh
0,2,932,80.12,2,1,3,0.003219,0.085966,0.666667,0,0.987175,0.012826,0
1,2,622,60.75,11,5,4,0.012862,0.097669,1.375000,0,0.987175,0.012826,0


In [67]:
# predictions unfortunately are shuffled, use X columns (so without 'fraud' column) as key for classification
val_df_i = val_df.set_index(list(test_df.columns), verify_integrity=True)
test_df_i = test_df.set_index(list(test_df.columns), verify_integrity=True)

val_pred1_df_i = val_pred1_df.set_index(list(test_df.columns), verify_integrity=True)
val_pred2_df_i = val_pred2_df.set_index(list(test_df.columns), verify_integrity=True)
test_pred_df_i = test_pred_df.set_index(list(test_df.columns), verify_integrity=True)

# as we can see it is still another sorting but indices allow us to join the dataframes now.
display(val_df_i.head(3))
display(val_pred1_df_i.head(3))

fraud
trustLevel totalScanTimeInSeconds grandTotal lineItemVoids scansWithoutRegistration quantityModifications scannedLineItemsPerSecond valuePerSecond lineItemVoidsPerPosition       
6          1504                   89.37      8             5                        3                     0.018617                  0.059422       0.285714                      0
           128                    63.32      5             4                        4                     0.226562                  0.494688       0.172414                      0
3          1675                   21.22      10            6                        4                     0.001791                  0.012669       3.333333                      0

fraud_pred  \
trustLevel totalScanTimeInSeconds grandTotal lineItemVoids scansWithoutRegistration quantityModifications scannedLineItemsPerSecond valuePerSecond lineItemVoidsPerPosition               
2          932                    80.12      2             1                        3                     0.003219                  0.085966       0.666667                           0   
           622                    60.75      11            5                        4                     0.012862                  0.097669       1.375000                           0   
1          1264                   42.18      0             6                        4                     0.014241                  0.033370       0.000000                           0   

                                                                                                                                                                             fraud_0_score  \
trustLevel totalScanTimeInSeconds grandTotal lineItemVoids scansWithoutRegistration quantityModifications scannedLineItemsPerSecond valuePerSecond lineItemVoidsPerPosition                  
2          932                    80.12      2             1                        3                     0.003219                  0.085966       0.666667                       0.987175   
           622                    60.75      11            5                        4                     0.012862                  0.097669       1.375000                       0.987175   
1          1264                   42.18      0             6                        4                     0.014241                  0.033370       0.000000                       0.815823   

                                                                                                                                                                             fraud_1_score  \
trustLevel totalScanTimeInSeconds grandTotal lineItemVoids scansWithoutRegistration quantityModifications scannedLineItemsPerSecond valuePerSecond lineItemVoidsPerPosition                  
2          932                    80.12      2             1                        3                     0.003219                  0.085966       0.666667                       0.012826   
           622                    60.75      11            5                        4                     0.012862                  0.097669       1.375000                       0.012826   
1          1264                   42.18      0             6                        4                     0.014241                  0.033370       0.000000                       0.184177   

                                                                                                                                                                             fraud_pred_thresh  
trustLevel totalScanTimeInSeconds grandTotal lineItemVoids scansWithoutRegistration quantityModifications scannedLineItemsPerSecond valuePerSecond lineItemVoidsPerPosition                     
2          932                    80.12      2             1                        3                     0.003219                  0.085966       0.666667                                  0  
           622                    60.75      11            5                        4                     0.012862                  0.097669       1.375000                                  0  
1          1264                   42.18      0             6                        4                     0.014241                  0.033370       0.000000                                  0

In [68]:
# join predictions to original val data
val_pred1_df_c = val_df_i.join(val_pred1_df_i)
val_pred2_df_c = val_df_i.join(val_pred2_df_i)
test_pred_df_c = test_df_i.join(test_pred_df_i)

val_pred1_df_c.head()

fraud  \
trustLevel totalScanTimeInSeconds grandTotal lineItemVoids scansWithoutRegistration quantityModifications scannedLineItemsPerSecond valuePerSecond lineItemVoidsPerPosition          
6          1504                   89.37      8             5                        3                     0.018617                  0.059422       0.285714                      0   
           128                    63.32      5             4                        4                     0.226562                  0.494688       0.172414                      0   
3          1675                   21.22      10            6                        4                     0.001791                  0.012669       3.333333                      0   
1          1753                   14.41      5             1                        1                     0.006275                  0.008220       0.454545                      0   
5          157                    33.89      11            3                        3                     0.146497                  0.215860       0.478261                      0   

                                                                                                                                                                             fraud_pred  \
trustLevel totalScanTimeInSeconds grandTotal lineItemVoids scansWithoutRegistration quantityModifications scannedLineItemsPerSecond valuePerSecond lineItemVoidsPerPosition               
6          1504                   89.37      8             5                        3                     0.018617                  0.059422       0.285714                           0   
           128                    63.32      5             4                        4                     0.226562                  0.494688       0.172414                           0   
3          1675                   21.22      10            6                        4                     0.001791                  0.012669       3.333333                           0   
1          1753                   14.41      5             1                        1                     0.006275                  0.008220       0.454545                           0   
5          157                    33.89      11            3                        3                     0.146497                  0.215860       0.478261                           0   

                                                                                                                                                                             fraud_0_score  \
trustLevel totalScanTimeInSeconds grandTotal lineItemVoids scansWithoutRegistration quantityModifications scannedLineItemsPerSecond valuePerSecond lineItemVoidsPerPosition                  
6          1504                   89.37      8             5                        3                     0.018617                  0.059422       0.285714                       0.991176   
           128                    63.32      5             4                        4                     0.226562                  0.494688       0.172414                       0.991176   
3          1675                   21.22      10            6                        4                     0.001791                  0.012669       3.333333                       0.993208   
1          1753                   14.41      5             1                        1                     0.006275                  0.008220       0.454545                       0.966544   
5          157                    33.89      11            3                        3                     0.146497                  0.215860       0.478261                       0.993208   

                                                                                                                                                                             fraud_1_score  \
trustLevel totalScanTimeInSeconds grandTotal lineItemVoids scansWit

In [72]:
# calculate scores
def score_function(y_true, y_pred):
  dmc = np.sum(metrics.confusion_matrix(y_true, y_pred)*np.array([[0, -25],[ -5, 5]])) #sklearn gives [[tn,fp],[fn,tp]]
  return (#0 if all(y_pred == 0) else metrics.fbeta_score(y_true, y_pred, beta=2),
          dmc, 
          dmc/len(y_pred), #comparable relative score, the higher the better.
          metrics.confusion_matrix(y_true, y_pred).tolist(),
          0 if all(y_pred == 0) else metrics.fbeta_score(y_true, y_pred, beta=0.5172))

print("val with groundtruth   ",score_function(val_pred1_df_c['fraud'].values, val_pred1_df_c["fraud"].values))
print("val 1 with 0.5   ",score_function(val_pred1_df_c['fraud'].values, val_pred1_df_c["fraud_pred"].values))
print("val 1 with thresh",score_function(val_pred1_df_c['fraud'].values, val_pred1_df_c["fraud_pred_thresh"].values))
print("val 2 with 0.5   ",score_function(val_pred2_df_c['fraud'].values, val_pred2_df_c["fraud_pred"].values))
print("val 2 with thresh",score_function(val_pred2_df_c['fraud'].values, val_pred2_df_c["fraud_pred_thresh"].values))
print("test             ","no ground truth available, TODO: Compare distributions")

val with groundtruth    (115, 0.3058510638297872, [[353, 0], [0, 23]], 1.0)
val 1 with 0.5    (115, 0.3058510638297872, [[353, 0], [0, 23]], 1.0)
val 1 with thresh (-35, -0.09308510638297872, [[353, 0], [15, 8]], 0.7164836794317928)
val 2 with 0.5    (115, 0.3058510638297872, [[353, 0], [0, 23]], 1.0)
val 2 with thresh (-35, -0.09308510638297872, [[353, 0], [15, 8]], 0.7164836794317928)
test              no ground truth available, TODO: Compare distributions


In [71]:
# compare test set:
test_pred_df_c.head(10)

fraud_0_score  \
trustLevel totalScanTimeInSeconds grandTotal lineItemVoids scansWithoutRegistration quantityModifications scannedLineItemsPerSecond valuePerSecond lineItemVoidsPerPosition                  
4          467                    88.48      4             8                        4                     0.014989                  0.189465       0.571429                       1.000000   
3          1004                   58.99      7             6                        1                     0.026892                  0.058755       0.259259                       1.000000   
1          162                    14.00      4             5                        4                     0.006173                  0.086420       4.000000                       1.000000   
5          532                    84.79      9             3                        4                     0.026316                  0.159380       0.642857                       1.000000   
           890                    42.16      4             0                        0                     0.021348                  0.047371       0.210526                       1.000000   
           1072                   12.67      3             4                        1                     0.019590                  0.011819       0.142857                       1.000000   
3          259                    93.75      0             7                        0                     0.100386                  0.361969       0.000000                       1.000000   
2          1528                   47.35      2             9                        5                     0.009817                  0.030988       0.133333                       0.971603   
6          816                    80.89      9             4                        0                     0.017157                  0.099130       0.642857                       1.000000   
4          16                     31.91      7             7                        4                     1.312500                  1.994375       0.333333                       1.000000   

                                                                                                                                                                             fraud_1_score  \
trustLevel totalScanTimeInSeconds grandTotal lineItemVoids scansWithoutRegistration quantityModifications scannedLineItemsPerSecond valuePerSecond lineItemVoidsPerPosition                  
4          467                    88.48      4             8                        4                     0.014989                  0.189465       0.571429                   1.228802e-15   
3          1004                   58.99      7             6                        1                     0.026892                  0.058755       0.259259                   4.509405e-10   
1          162                    14.00      4             5                        4                     0.006173                  0.086420       4.000000                   0.000000e+00   
5          532                    84.79      9             3                        4                     0.026316                  0.159380       0.642857                   4.408285e-16   
           890                    42.16      4             0                        0                     0.021348                  0.047371       0.210526                   1.028180e-14   
           1072                   12.67      3             4                        1                     0.019590                  0.011819       0.142857                   6.953312e-12   
3          259                    93.75      0             7                        0                     0.100386                  0.361969       0.000000                   6.456098e-14   
2          1528                   47.35      2             9                        5                     0.009817                  0.030988       0.133333                   2.839732